<a href="https://colab.research.google.com/github/Nani2421/REAL-TIME-THEFT-DETECTION/blob/main/yolov10n_custom_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'
print("W&B logging has been disabled.")

W&B logging has been disabled.


In [ ]:
!sed -i 's/torch.load(file, map_location="cpu")/torch.load(file, map_location="cpu", weights_only=False)/' /usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py

In [ ]:
!wget -P '/content/drive/MyDrive/yolov10_weights' https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt

--2025-10-01 13:24:00--  https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/804788522/411e0d4f-1023-40ad-bfdd-c99f0dddb73b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-01T14%3A01%3A30Z&rscd=attachment%3B+filename%3Dyolov10n.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-01T13%3A00%3A58Z&ske=2025-10-01T14%3A01%3A30Z&sks=b&skv=2018-11-09&sig=TQ6huPhibUNPXKPtxqiQ6irbY4EX6G0vQstjWQXRgO8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1OTMyNTM0MCwibmJmIjoxNzU5MzI1MDQwLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmUu

In [ ]:
# Replace the placeholder path with the actual location of your zipped file.
# The dataset will be unzipped into a folder called 'yolov10_data' in the local Colab environment.
!unzip -q '/content/drive/MyDrive/yolov10_training/archive.zip' -d '/content/yolov10_data'

In [ ]:
%%writefile /content/yolov10_data/data.yaml
path: /content/yolov10_data
train: train/images
val: valid/images
test: test/images

nc: 2
names: ['human', 'suspicion']

Overwriting /content/yolov10_data/data.yaml


In [ ]:
!yolo detect train \
    data=/content/yolov10_data/data.yaml \
    model=/content/drive/MyDrive/yolov10_weights/yolov10n.pt \
    epochs=40 \
    imgsz=640 \
    project=/content/drive/MyDrive/yolov10_training_results \
    name=yolov10n_results \
    save_json=False

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov10_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, kera

In [ ]:
!yolo detect predict \
    model=/content/drive/MyDrive/yolov10_training_results/yolov10n_results/weights/best.pt \
    source='/content/thief-stealing-handbag-27891534.webp' \
    save=True

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv10n summary (fused): 102 layers, 2,265,558 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /content/thief-stealing-handbag-27891534.webp: 512x640 1 suspicion, 200.0ms
Speed: 5.1ms preprocess, 200.0ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)
Results saved to /content/runs/detect/predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo detect train \
    data=/content/yolov10_data/data.yaml \
    model=/content/drive/MyDrive/yolov10_training_results/yolov10n_results/weights/last.pt \
    epochs=60 \
    imgsz=640 \
    project=/content/drive/MyDrive/yolov10_training_results \
    name=yolov10n_results_continued \
    save_json=False

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov10_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, kera